In [1]:
!unzip "/content/data.zip"

Archive:  /content/data.zip
   creating: Test Data/Bird/
   creating: Test Data/Bird/GIB_AUG/
  inflating: Test Data/Bird/GIB_AUG/aug_download (1)_0_1282.jpg  
  inflating: Test Data/Bird/GIB_AUG/aug_download (1)_0_1347.jpg  
  inflating: Test Data/Bird/GIB_AUG/aug_download (1)_0_2011.jpg  
  inflating: Test Data/Bird/GIB_AUG/aug_download (1)_0_2501.jpg  
  inflating: Test Data/Bird/GIB_AUG/aug_download (1)_0_2661.jpg  
  inflating: Test Data/Bird/GIB_AUG/aug_download (1)_0_590.jpg  
  inflating: Test Data/Bird/GIB_AUG/aug_download (10)_0_1906.jpg  
  inflating: Test Data/Bird/GIB_AUG/aug_download (10)_0_2359.jpg  
  inflating: Test Data/Bird/GIB_AUG/aug_download (10)_0_4774.jpg  
  inflating: Test Data/Bird/GIB_AUG/aug_download (10)_0_5033.jpg  
  inflating: Test Data/Bird/GIB_AUG/aug_download (10)_0_5329.jpg  
  inflating: Test Data/Bird/GIB_AUG/aug_download (10)_0_5590.jpg  
  inflating: Test Data/Bird/GIB_AUG/aug_download (10)_0_8499.jpg  
  inflating: Test Data/Bird/GIB_AUG/aug_do

#Importing required Libraries

In [3]:
import os, gc, glob, random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras
from PIL import Image
from keras.layers import Dense,Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

#Using the InceptionV3 pretrained model

In [4]:
base_model = InceptionV3(input_shape=(224,224,3),include_top=False)

87910968/87910968 [==============================] - 5s 0us/step


#Freezing the weights in the pretrained model

In [5]:
for layer in base_model.layers:
  layer.trainable = False

#Flatening and adding dense layer


In [6]:
X = Flatten()(base_model.output)

#using sigmoid function as activation funciton
X = Dense(units = 6, activation = 'sigmoid')(X)

#Compiling the model

In [7]:
model = Model(base_model.input,X)

#using adam as the optimizer
#accuracy is set as the metrics to judge the performance of the model
model.compile(optimizer = 'adam',loss = keras.losses.binary_crossentropy,metrics=['accuracy'])

#summarizing the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

#loading the train and test dataset

In [20]:
#initializing the datagenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator()

#loading the train dataset from the directory
traindata= datagen.flow_from_directory(directory="/content/Train Data",target_size=(224,224),batch_size=100)

#loading the test dataset from the directory
testdata= datagen.flow_from_directory(directory="/content/Test Data",target_size=(224,224),batch_size=100)

Found 2916 images belonging to 6 classes.
Found 1427 images belonging to 6 classes.


In [17]:
#listing the found classes from train dataset
traindata.class_indices

{'Corpse_AUG': 0,
 'GIB_AUG': 1,
 'LS_Orchid_AUG': 2,
 'LS_Pangolin_AUG': 3,
 'SPS_AUG': 4,
 'SW_Deer_AUG': 5}

#Model Checkpoint and Early Stopping

In [18]:
# Creating a model checkpoint which monitors the accuracy of the model and saves the checkpoint
mc = ModelCheckpoint(filepath = "./model.h5",
                     monitor = 'accuracy',
                     verbose = 1,
                     save_best_only = True)

# Creating a earlystopping object which stop training once the model performance stops improving on a hold out validation dataset
es = EarlyStopping(monitor = "accuracy",
                   min_delta = 0.01,
                   verbose = 1)

call_back = [mc,es]

# Model fitting

In [25]:
# Fitting the model 
modelHistory = model.fit(traindata, steps_per_epoch=60,epochs = 30,callbacks=call_back,validation_data=testdata)

Epoch 1/30
30/60 [==============>...............] - ETA: 6s - loss: 1.1722 - accuracy: 0.9098


Epoch 1: accuracy improved from 0.86831 to 0.90981, saving model to ./model.h5
60/60 [==============================] - 13s 208ms/step - loss: 1.1722 - accuracy: 0.9098 - val_loss: 0.6965 - val_accuracy: 0.9636


#Exporting the model

In [21]:
# Exporting the model to json 
model_json = model.to_json()
with open("DigitalNaturalist.json", "w") as json_file:
    json_file.write(model_json)

# Exporting the model weights

model.save_weights("DigitalNaturalist")
print("Saved model to disk")

Saved model to disk


# Testing the model

In [30]:
#Testing the model 
predictions = ["Corpse Flower", 
               "Great Indian Bustard", 
               "Lady's slipper orchid", 
               "Pangolin", 
               "Spoon Billed Sandpiper", 
               "Seneca White Deer"
              ]


path = '/content/Test Data/GIB_AUG/aug_download (1)_0_2011.jpg'
ime = tf.keras.utils.load_img(path,target_size=(224,224))

i = tf.keras.preprocessing.image.img_to_array(ime)
i = preprocess_input(i)
input = np.array([i])
pred = model.predict(input)
predictions[np.argmax(pred)]

1/1 [==============================] - 0s 39ms/step


"Great Indian Bustard"